In [131]:
import pandas as pd
import numpy as np
from fuzzywuzzy import process, fuzz
from matplotlib import pyplot as plt
import seaborn as sns
import re

In [272]:
'''
High School Data
'''

# read in data
df = pd.read_csv('../data/exploring_data.csv')

# read in city of chicago zipcodes
zipcodes = pd.read_csv('../data/chicago_zipcodes.csv')

# convert to integers
df['s_zip']=pd.to_numeric(df['s_zip'], errors='coerce')
df.s_zip = df.s_zip.fillna(0)
df.s_zip = df.s_zip.astype(int)
df['homezip']=pd.to_numeric(df['homezip'], errors='coerce')
df.homezip = df.homezip.fillna(0)
df.homezip = df.homezip.astype(int)
zipcodes['ZIP']=pd.to_numeric(zipcodes['ZIP'], errors='coerce',downcast='integer')
zipcodes.ZIP = zipcodes.ZIP.astype(int)

# column for in chicago
df['chi_school']=0
df['notchi_school']=0
df['chi_home']=0
df['notchi_home']=0

df.loc[df['s_zip'].isin(zipcodes['ZIP']),'chi_school']=1
df.loc[~df['s_zip'].isin(zipcodes['ZIP']),'notchi_school']==1
df.loc[df['homezip'].isin(zipcodes['ZIP']),'chi_home']==1
df.loc[~df['homezip'].isin(zipcodes['ZIP']),'notchi_home']==1

df.columns

Index(['Unnamed: 0', 'instrument', 'gradyear', 'major', 'financial_aid',
       'race', 'gender', 'm_employ', 'm_job', 'd_employ', 'homezip', 'school',
       's_address', 's_state', 's_zip', 's_ensemble', 'pt', 'studentID',
       'caucasian', 'african_american', 'native_american', 'latino', 'asian',
       'other_race', 'multiracial', 'male', 'female', 'music', 'median_income',
       'mean_income', 'Pop', 'income_diff', 'uni', 'acceptance_rate',
       'uni_public', 'chi_school', 'notchi_school', 'chi_home', 'notchi_home'],
      dtype='object')

In [273]:
df.loc[df['s_zip'].isin(zipcodes['ZIP']),'chi_school']

11     0
44     0
51     0
59     0
67     0
89     0
94     0
104    0
107    0
114    0
116    0
121    0
122    0
131    0
133    0
140    0
141    0
172    0
178    0
183    0
186    0
189    0
193    0
197    0
211    0
233    0
240    0
242    0
253    0
258    0
      ..
306    0
316    0
344    0
347    0
352    0
354    0
360    0
368    0
373    0
374    0
388    0
391    0
393    0
400    0
402    0
404    0
407    0
418    0
422    0
423    0
426    0
432    0
437    0
441    0
444    0
448    0
450    0
451    0
453    0
467    0
Name: chi_school, Length: 68, dtype: int64

In [241]:
zipcodes.columns

Index(['ZIP'], dtype='object')

In [218]:
# """
# explore college data
# """

# # read in data
# df = pd.read_csv('../data/exploring_data.csv')
# uf = pd.read_csv('../data/colleges.csv')

# df['uni_close'] = 'none'
# df['uni_save'] = df['uni']

# #
# df['uni'] = df['uni'].fillna('none given')
# df['uni'] = df['uni'].str.replace('Uof','University of')
# df['uni'] = df['uni'].str.replace('U of','University of')
# df['uni'] = df['uni'].str.replace('Jacobs School of Music','')
# uf['displayName'] = uf['displayName'].fillna('none given')

# def stripnames(dataframe,instring,outstring):
#     dataframe.loc[df['uni'].str.replace(' ','').replace("'",'').replace('-','') \
#                   ==instring.replace(' ','').replace("'",'').replace('-',''),'uni']=outstring
#     return dataframe

# # # heres where I'm lining up the names
# df = stripnames(df,'Indiana University','Indiana University--Bloomington')
# df = stripnames(df,'NYU','New York University')
# df = stripnames(df,'Oberlin','Oberlin College')
# df = stripnames(df,'UIC','University of Illinois Chicago')
# df = stripnames(df,'UIUC','University of Illinois--Urbana Champaign')
# df = stripnames(df,'USC(So.Cal.)','University of Southern California')
# df = stripnames(df,'UofMichigan','University of Michigan')
# df = stripnames(df,'Peabody','Johns Hopkins University')
# df = stripnames(df,'University of I','University of Illinois--Urbana Champaign')
# df = stripnames(df,'NIU','Northern Illinois University')
# df = stripnames(df,'U.ofMinnesotaCarlsonSchoolofBusiness','University of Minnesota')
# df = stripnames(df,'NEC','New England Conservatory')
# df = stripnames(df,'MIT','Massachussetts Institute of Technology')
# df = stripnames(df,'Gap Year','none')
# df = stripnames(df,'Year off','none')
# df = stripnames(df,'The Juilliard School','Julliard School')
# df = stripnames(df,'IUBloomington,JacobsSchoolofMusic','Indiana University--Bloomington')
# df = stripnames(df,'IUJacobsSchoolofMusic','Indiana University--Bloomington')
# df = stripnames(df,'HarvardCollege','Harvard University')
# df = stripnames(df,'Undecided','none')
# df = stripnames(df,'NotgoingtoCollege','none')
# df = stripnames(df,'IndianaUniversityJacobsSchoolofMusic','Indiana University--Bloomington')
# df = stripnames(df,'IndianaUniversity,JacobsSchoolofMusic','Indiana University--Bloomington')
# df = stripnames(df,'IndianaUniversity(JacobsSchoolofMusic)','Indiana University--Bloomington')
# df = stripnames(df,'JacobsSchoolofMusic,atIndianaUniversity','Indiana University--Bloomington')
# df = stripnames(df,'ClarendonHillsMiddleSchool','none')
# df = stripnames(df,'IU','Indiana University--Bloomington')
# df = stripnames(df,'LawrenceConservatory','Lawrence University')
# df = stripnames(df,'IndianaUniversity','Indiana University--Bloomington')
# df = stripnames(df,'WesternIllinoisUniversity','WesternIllinois')
# df = stripnames(df,'UniversityofIllinois','University of Illinois--Urbana Champaign')
# df = stripnames(df,',atIndianaUniversity','Indiana University--Bloomington')
# df = stripnames(df,'X','X')
# df = stripnames(df,'X','X')
# df = stripnames(df,'X','X')
# df = stripnames(df,'X','X')
# df = stripnames(df,'X','X')

# for index, row in df.iterrows():
#     df.loc[index, 'uni_close']=process.extractOne(row['uni'],uf['displayName'])[0]
    


# # for index, row in df.iterrows():
# #     actual_uni = row['uni'].replace(' ','').replace("'",'').replace('-','')
# #     closest_uni = row['uni_close'].replace(' ','').replace("'",'').replace('-','')
# #     if 90 < fuzz.ratio(actual_uni,closest_uni) < 100:
# #         print(actual_uni.strip(), ' AND ',closest_uni.strip())

Unnamed: 0            int64
instrument           object
gradyear              int64
major                object
financial_aid       float64
race                 object
gender               object
m_employ             object
m_job                object
d_employ             object
homezip             float64
school               object
s_address            object
s_state              object
s_zip                object
s_ensemble           object
pt                   object
studentID             int64
caucasian             int64
african_american      int64
native_american       int64
latino                int64
asian                 int64
other_race            int64
multiracial           int64
male                  int64
female                int64
music                 int64
median_income       float64
mean_income         float64
Pop                  object
income_diff         float64
uni                  object
acceptance_rate     float64
uni_public            int64
dtype: object